In [ ]:
import datetime
today = datetime.date.today()
print(today.day)

6


In [ ]:
! pip install google-api-python-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

In [ ]:
scopes = ['https://www.googleapis.com/auth/calendar']

In [ ]:
CLIENT_SECRETS_FILE = "client_secret.json"
flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, scopes=scopes)
credentials = flow.run_console()

FileNotFoundError: ignored

In [ ]:
print(credentials)

In [ ]:
import pickle

In [ ]:
pickle.dump(credentials, open("token.pkl","wb"))

NameError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
credentials = pickle.load(open("token.pkl","rb"))

In [ ]:
service = build("calendar","v3", credentials=credentials)

Get My Calendars

In [ ]:
result = service.calendarList().list().execute()

In [ ]:
calendar_id = result['items'][0]['id']

Get My Calendar Events


In [ ]:
result = service.events().list(calendarId=calendar_id).execute()

In [17]:
event = result['items'][0]

**Formatting the Event Date**

In [ ]:
import datetime

eventDate = event['start']['dateTime'][5:10]
tdelta1 = datetime.timedelta(days=2) 
tdelta2 = datetime.timedelta(days=10)

eventStart = event['start']['dateTime'][11:16]
start_hour = int(event['start']['dateTime'][11:13])
start_minutes = event['start']['dateTime'][14:16]
if start_hour > 12:
    start_hour = start_hour - 12

eventStart = f'{start_hour}:{start_minutes}'
#print(eventStart)


eventEnd = event['end']['dateTime'][11:16]
end_hour = int(event['end']['dateTime'][11:13])
end_minutes = event['end']['dateTime'][14:16]
if end_hour > 12:
  end_hour = end_hour - 12
  
eventEnd = f'{end_hour}:{end_minutes}'
summary = event['summary']
print(f'\t{summary}, {eventStart} to {eventEnd}')
summary =event['summary']
datetime = event['start']['dateTime'][5:10]
print(f'\t{summary}, {datetime} -> Last Stretch!')
#print(eventEnd)


	Midterm, 2:00 to 3:00
	Midterm, 11-02 -> Last Stretch!


Get my calendars

In [18]:
page_token = None
while True:
  events = service.events().list(calendarId=calendar_id, pageToken=page_token).execute()
  print("Your Daily at a Glance:")
  for event in events['items']:
    eventStart = event['start']['dateTime'][11:16]
    start_hour = int(event['start']['dateTime'][11:13])
    start_minutes = event['start']['dateTime'][14:16]
    if start_hour > 12:
        start_hour = start_hour - 12

    eventStart = f'{start_hour}:{start_minutes}'
    #print(eventStart)


    eventEnd = event['end']['dateTime'][11:16]
    end_hour = int(event['end']['dateTime'][11:13])
    end_minutes = event['end']['dateTime'][14:16]
    if end_hour > 12:
      end_hour = end_hour - 12
  
    eventEnd = f'{end_hour}:{end_minutes}'
    #print(eventEnd)
    # events today
    # replace 6 with today var
    if int(event['start']['dateTime'][8:10]) == today.day:
        summary = event['summary']
        print(f'   {summary}, {eventStart} to {eventEnd}')
  print("Upcoming tests:")
  for event in events['items']:
    # tests within next two days
    # replace 6 with today var
    # replace 8 with today var + 2
    if (("test" in (event['summary']).lower()) &
        (int(event['start']['dateTime'][8:10]) > today.day) &
        (int(event['start']['dateTime'][8:10]) <= (today+tdelta1).day)):
        summary =event['summary']
        date = event['start']['dateTime'][5:10]
        print(f'   {summary}, {date} -> Last Stretch!')
    # tests within next third day to tenth day from today
    # replace 8 with today var + 2
    # replace 16 with today var + 10
    # don't worry about num > 31 edge case because calendar will never ouput it
    elif (("test" in (event['summary']).lower()) &
        (int(event['start']['dateTime'][8:10]) > (today+tdelta1).day) &
         (int(event['start']['dateTime'][8:10]) <= (today+tdelta2).day)):
        summary =event['summary']
        date = event['start']['dateTime'][5:10]
        print(f'   {summary}, {date} -> Start Studying!')
  page_token = events.get('nextPageToken')
  if not page_token:
    break

Your Daily at a Glance:
   Random6, 9:00 to 10:00
   Lunch with Mom, 1:00 to 2:00
Upcoming tests:
   Test8, 11-08 -> Last Stretch!
   Test11, 11-11 -> Start Studying!
